#** Machine learning IB-2024** #** Home task 1.** #Regression, KNN, LinearRegression.

In this homework we will build models to predict the price of an apartment in Russia. below is a description of some columns of the data set. date - date of publication price - price in rubles level- floor, on which is the apartment levels - number of floors in the apartment rooms - number of rooms in the apartment. If value -1, then the apartment is considered apartments. area - area of the apartment. kitchen_area - area of the kitchen. geo_lat - Latitude geo_lon - Longitude building_type - building material. 0 - Don't know. 1 - Other. 2 - Panel. 3 - Monolithic. 4 - Brick. 5 - Blocky. 6 - Wooden

Part 0 – Start of work

To start working with the data, we import the libraries that will be needed in this task.

In [2]:
import math
import pandas as pd
import numpy as np
import matplotlib as plt
import sklearn
import seaborn as sns

Download the folium library to display the data on the map by coordinates.

In [3]:
!pip install folium


We delete our data from the archive.

Download the data from the CSV file into the data frame.

In [83]:
df = pd.read_csv("/content/input_data.csv", delimiter=';')
# Since the data set is very large and it is very difficult to work with it due to time limitation, we leave the first 1000 lines.
df = df.head(1000)
df

,date,price,level,levels,rooms,area,kitchen_area,geo_lat,geo_lon,building_type,object_type,postal_code,street_id,id_region,house_id
0,2021-01-01,2451300,15,31,1,30.3,0.0,56.780112,60.699355,0,2,620000.0,NaN,66,1632918.0
1,2021-01-01,1450000,5,5,1,33.0,6.0,44.608154,40.138381,0,0,385000.0,NaN,1,NaN
2,2021-01-01,10700000,4,13,3,85.0,12.0,55.540060,37.725112,3,0,142701.0,242543.0,50,681306.0
3,2021-01-01,3100000,3,5,3,82.0,9.0,44.608154,40.138381,0,0,385000.0,NaN,1,NaN
4,2021-01-01,2500000,2,3,1,30.0,9.0,44.738685,37.713668,3,2,353960.0,439378.0,23,1730985.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,2021-01-01,1540000,3,5,2,49.0,7.0,52.535440,85.250308,2,0,659330.0,584206.0,22,2729958.0
996,2021-01-01,12000000,1,9,2,47.3,7.1,55.743786,37.687728,4,0,109544.0,479669.0,77,1537362.0
997,2021-01-01,2650000,2,3,2,43.5,6.0,56.748803,38.866829,4,0,152020.0,368469.0,76,983924.0
998,2021-01-01,2880000,5,5,3,62.0,6.0,58.036182,56.036620,2,0,614023.0,NaN,59,NaN


Show the coordinates of our buildings on the map.

In [51]:
import folium
from IPython.display import display

map_df = df.loc[:1000]

m = folium.Map(location=[55.751244, 37.618423], zoom_start=10)

# List of points with width and length
lats = map_df['geo_lat'].loc[:1000]
longs = map_df['geo_lon'].loc[:1000]
# Adding points to the map
for point in zip(lats, longs):
    folium.Marker(
        location=[point[0], point[1]]
    ).add_to(m)

display(m)

Part 1: Preparing data for processing by machine learning models.

**0.5 Ball**. The geography of our observations in the data set is extremely large. However, we know that the cost of apartments in Moscow and St. Petersburg is much higher than the average in Russia. Let's make signs that show whether the apartment is 20 kilometers from the centre of Moscow or whether the apartment is 20 kilometers from the centre of St. Petersburg. Create two signs is_Moscow and is_Saint_Peterburg. To find the distance by the coordinates use the function haversine_distance.

In [85]:
def haversine_distance(lat1, lon1, lat2, lon2):
    R = 6371  # Earth's radius in kilometers
    lat1, lon1, lat2, lon2 = map(np.radians, [lat1, lon1, lat2, lon2])
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = np.sin(dlat / 2)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon / 2)**2
    c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1 - a))
    return R * c

moscow_center = (55.7558, 37.6173)
saint_petersburg_center = (59.9343, 30.3351)

df['is_Moscow'] = df.apply(
    lambda row: haversine_distance(row['geo_lat'], row['geo_lon'], moscow_center[0], moscow_center[1]) <= 20,
    axis=1
)

df['is_Saint_Petersburg'] = df.apply(
    lambda row: haversine_distance(row['geo_lat'], row['geo_lon'], saint_petersburg_center[0], saint_petersburg_center[1]) <= 20,
    axis=1
)

print(df)

           date     price  level  levels  rooms  area  kitchen_area  \
0    2021-01-01   2451300     15      31      1  30.3           0.0   
1    2021-01-01   1450000      5       5      1  33.0           6.0   
2    2021-01-01  10700000      4      13      3  85.0          12.0   
3    2021-01-01   3100000      3       5      3  82.0           9.0   
4    2021-01-01   2500000      2       3      1  30.0           9.0   
..          ...       ...    ...     ...    ...   ...           ...   
995  2021-01-01   1540000      3       5      2  49.0           7.0   
996  2021-01-01  12000000      1       9      2  47.3           7.1   
997  2021-01-01   2650000      2       3      2  43.5           6.0   
998  2021-01-01   2880000      5       5      3  62.0           6.0   
999  2021-01-01   3960000      6       6      3  85.0          13.0   

       geo_lat    geo_lon  building_type  object_type  postal_code  street_id  \
0    56.780112  60.699355              0            2     620000.0

**0.5 Ball**. In our data set there are signs that we can theoretically use, such as postal_code, but we will do this within the framework of homework very-very long. Therefore it is proposed to remove unnecessary signs from the data frame. We will remove geo_lat, geo_lon, object_type, postal_code, street_id, id_region, house_id.

In [86]:
df.drop(columns=['geo_lat', 'geo_lon', 'object_type', 'postal_code', 'street_id', 'id_region', 'house_id'], inplace=True)
df

,date,price,level,levels,rooms,area,kitchen_area,building_type,is_Moscow,is_Saint_Petersburg
0,2021-01-01,2451300,15,31,1,30.3,0.0,0,False,False
1,2021-01-01,1450000,5,5,1,33.0,6.0,0,False,False
2,2021-01-01,10700000,4,13,3,85.0,12.0,3,False,False
3,2021-01-01,3100000,3,5,3,82.0,9.0,0,False,False
4,2021-01-01,2500000,2,3,1,30.0,9.0,3,False,False
...,...,...,...,...,...,...,...,...,...,...
995,2021-01-01,1540000,3,5,2,49.0,7.0,2,False,False
996,2021-01-01,12000000,1,9,2,47.3,7.1,4,True,False
997,2021-01-01,2650000,2,3,2,43.5,6.0,4,False,False
998,2021-01-01,2880000,5,5,3,62.0,6.0,2,False,False


What columns are categorical? What numbers are? Categorical: building_type, rooms Numbers: date, price, level, levels, area, kitchen_area Let’s encode categorical characters using OneHot-Encoding. https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OneHotEncoder.html

In [87]:
from sklearn.preprocessing import OneHotEncoder

categorical_features = ['building_type', 'rooms']

encoder = OneHotEncoder(sparse_output=False)

one_hot_encoded = encoder.fit_transform(df[categorical_features])

one_hot_df = pd.DataFrame(one_hot_encoded, columns=encoder.get_feature_names_out(categorical_features), index=df.index)
df_encoded = pd.concat([df, one_hot_df], axis=1)

df = df_encoded


df_encoded

,date,price,level,levels,rooms,area,kitchen_area,building_type,is_Moscow,is_Saint_Petersburg,...,building_type_5,building_type_6,rooms_-1,rooms_1,rooms_2,rooms_3,rooms_4,rooms_5,rooms_6,rooms_7
0,2021-01-01,2451300,15,31,1,30.3,0.0,0,False,False,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2021-01-01,1450000,5,5,1,33.0,6.0,0,False,False,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2021-01-01,10700000,4,13,3,85.0,12.0,3,False,False,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
3,2021-01-01,3100000,3,5,3,82.0,9.0,0,False,False,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
4,2021-01-01,2500000,2,3,1,30.0,9.0,3,False,False,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,2021-01-01,1540000,3,5,2,49.0,7.0,2,False,False,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
996,2021-01-01,12000000,1,9,2,47.3,7.1,4,True,False,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
997,2021-01-01,2650000,2,3,2,43.5,6.0,4,False,False,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
998,2021-01-01,2880000,5,5,3,62.0,6.0,2,False,False,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0


**0.5 Ball**. We work with numerical characters: 1. Add two characters to your dataset: the number of days from the day of the first observation (the difference between the dates of the ads). Maybe for the price forecast the floor is not as important as the ratio of the floor to the number of floors in the house, add this character. After adding a new character the date column can be removed. 2. The numerical characters may have different orders. Let us compare the numerical characters using StandartScaller https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html.

In [88]:
df['date'] = pd.to_datetime(df['date'])
df['days_since_first'] = (df['date'] - df['date'].min()).dt.days
df.drop(columns =['date'], inplace=True)
df['levels_ratio'] = (df['level'] / df['levels'])
df


,price,level,levels,rooms,area,kitchen_area,building_type,is_Moscow,is_Saint_Petersburg,building_type_0,...,rooms_-1,rooms_1,rooms_2,rooms_3,rooms_4,rooms_5,rooms_6,rooms_7,days_since_first,levels_ratio
0,2451300,15,31,1,30.3,0.0,0,False,False,1.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.483871
1,1450000,5,5,1,33.0,6.0,0,False,False,1.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0,1.000000
2,10700000,4,13,3,85.0,12.0,3,False,False,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0,0.307692
3,3100000,3,5,3,82.0,9.0,0,False,False,1.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0,0.600000
4,2500000,2,3,1,30.0,9.0,3,False,False,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.666667
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,1540000,3,5,2,49.0,7.0,2,False,False,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0,0.600000
996,12000000,1,9,2,47.3,7.1,4,True,False,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0,0.111111
997,2650000,2,3,2,43.5,6.0,4,False,False,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0,0.666667
998,2880000,5,5,3,62.0,6.0,2,False,False,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0,1.000000


In [89]:
# Let’s check on NaN.
df.isna().sum(axis=0)

,0
price,0
level,0
levels,0
rooms,0
area,0
kitchen_area,0
building_type,0
is_Moscow,0
is_Saint_Petersburg,0
building_type_0,0


In [90]:
# Before comparing the data, we will divide it into training and test samples so that the model does not have any information about the test sample.

x = df.drop(columns=['price']).values
y = df['price'].values

x_train, x_test, y_train, y_test = train_test_split(x, y, shuffle=True, test_size=0.2, random_state=44)




In [91]:
# Now can be compared.
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
scaler.fit(x_train)


x_train_scaled = scaler.transform(x_train)
x_test_scaled = scaler.transform(x_test)





In [92]:

print('Before the scalation:',x_train.mean(axis=0))
print('After the scalation:',x_train_scaled.mean(axis=0))

До масштабирования:  [6.22625 11.3375 1.6825 54.05291249999996 -5.858374999999999 2.24625
 0.08625 0.04875 0.30125 0.02625 0.18 0.15875 0.2925 0.03375 0.0075
 0.09375 0.32125 0.34875 0.19625 0.035 0.0025 0.00125 0.00125 0.0
 0.5770493830380965]
После масштабирования:  [-6.57807142e-17  1.76803017e-16 -1.76941795e-16  1.48117629e-15
 -4.87804241e-17  1.47347412e-16 -7.49400542e-17 -7.59808882e-17
  4.43256543e-16 -3.10584891e-16 -4.88498131e-17 -4.23272528e-17
  9.95037386e-17 -4.44436155e-16  4.80969431e-16  2.39530618e-16
  1.72084569e-16  2.70616862e-17  7.20257187e-17  3.73243103e-16
  3.39051703e-16  1.85129689e-16 -7.22512328e-17  0.00000000e+00
  3.93574062e-16]


Realize the KNNRegressor class, which should make regression by the method to the nearest neighbors.

In [106]:
import numpy as np
from scipy.spatial.distance import cdist

class KNNRegressor:
    def __init__(self, n_neighbors=3, metric='euclidean'):
        self.n_neighbors = n_neighbors
        self.metric = metric

    def fit(self, X, y):
        self.X_train = X
        self.y_train = y

    def predict(self, X):
        predictions = []
        for x in X:
            distances = cdist([x], self.X_train, metric=self.metric).flatten()
            nearest_indices = np.argsort(distances)[:self.n_neighbors]
            nearest_values = self.y_train[nearest_indices]
            prediction = np.mean(nearest_values)
            predictions.append(prediction)
        return np.array(predictions)


knn = KNNRegressor(n_neighbors=3, metric='euclidean')
knn.fit(x_train_scaled, y_train)


In [107]:
predictions = knn.predict(x_test_scaled)
print("The forecast:", predictions)

Предсказания: [ 3700066.66666667  4350000.          4492483.66666667  9417783.33333333
  5481781.66666667  7009150.33333333  2330000.          5133393.33333333
  4810000.          4597452.          4340000.         11250000.
  2650000.          2750000.          1583333.33333333  5442625.
  2750000.          2716666.66666667  2246666.66666667  6433408.
  8220243.33333333  8411760.          4610000.          3460666.66666667
  4373130.33333333  3544000.          1666666.66666667  3544000.
  5333333.33333333  2163333.33333333  2806666.66666667  7192890.66666667
  2403573.33333333   863000.          2530000.          1745666.66666667
  2100000.          2833333.33333333 12597670.33333333  2250000.
  1233333.33333333  7331152.66666667  6942948.          2806666.66666667
  3466666.66666667  2040000.          4466666.66666667  5178333.33333333
  2586666.66666667  1803333.33333333  4390000.          7572073.33333333
  4223333.33333333 15666666.66666667   863000.          7266666.66666667
  21

In [43]:
from sklearn.neighbors import KNeighborsRegressor



In [105]:
# In general, everything can be done with the help of the sklearn library.
model = KNeighborsRegressor(n_neighbors = 3)
model.fit(x_train_scaled, y_train)
y_pred = model.predict(x_test_scaled)

In [103]:
y_pred

array([ 3864040. ,  4008628. ,  5795490.2,  7043870. ,  5213706.6,
        6169568.4,  3354910.6,  4700036. ,  4826000. ,  4894810.6,
        4262000. , 10110000. ,  2484000. ,  2989800. ,  2150000. ,
        4115891.2,  2989800. ,  2412000. ,  1900000. ,  5127844.8,
        6547146. ,  9007056. ,  4506000. ,  2868200. ,  5210186.2,
        3026400. ,  1880000. ,  3176400. ,  4643400. ,  2698000. ,
        2734000. ,  5153734.4,  2268144. ,  1267800. ,  2508000. ,
        2137400. ,  2400000. ,  2760000. , 15314202.2,  2300000. ,
        1930000. ,  6587720.8,  5748164.8,  2518000. ,  3610000. ,
        2658000. ,  4479999.8,  3892948.4,  2282000. ,  1678000. ,
        4714000. ,  7679526. ,  4992000. , 12710000. ,  1377800. ,
        7310000. ,  2342144. ,  4912000. ,  7636220. ,  3558000. ,
        3247732.8,  2878120. , 29085295.4,  2100000. ,  3080000. ,
        4710000. ,  2079560. ,  1938000. ,  3321200. ,  2474100. ,
        4972936. ,  2484000. ,  4710000. ,  3102000. ,  196000

**3 Balls**. Realize the LinearRegression class that supports gradient leaks SGD, Momentum, AdaGrad. Use gradient to optimize the MSE loss function.

In [110]:
class LinearRegression:
    def __init__(self, learning_rate=0.01, optimization='SGD', epsilon=1e-8, decay_rate=0.9, max_iter=1000):
        self.learning_rate = learning_rate
        self.optimization = optimization
        self.epsilon = epsilon
        self.decay_rate = decay_rate
        self.max_iter = max_iter
        self.weights = None
        self.bias = None
        self.velocity = None
        self.cache = None

    def fit(self, X, y):
        m, n = X.shape
        self.weights = np.zeros(n)
        self.bias = 0
        self.velocity = np.zeros(n)
        self.cache = np.zeros(n)

        for iteration in range(self.max_iter):
            gradients = self._compute_gradients(X, y)
            self._update_parameters(gradients)

    def _compute_gradients(self, X, y):
        m = X.shape[0]
        predictions = self.predict(X)
        error = predictions - y
        dw = (1 / m) * np.dot(X.T, error)
        db = (1 / m) * np.sum(error)
        return dw, db

    def _update_parameters(self, gradients):
        dw, db = gradients

        if self.optimization == 'SGD':
            self.weights -= self.learning_rate * dw
            self.bias -= self.learning_rate * db

        elif self.optimization == 'Momentum':
            self.velocity = self.decay_rate * self.velocity - self.learning_rate * dw
            self.weights += self.velocity
            self.bias -= self.learning_rate * db

        elif self.optimization == 'AdaGrad':
            self.cache += dw ** 2
            self.weights -= self.learning_rate * dw / (np.sqrt(self.cache) + self.epsilon)
            self.bias -= self.learning_rate * db

    def predict(self, X):
        return np.dot(X, self.weights) + self.bias

model = LinearRegression(learning_rate=0.01, optimization='SGD', max_iter=1000)
model.fit(x_train_scaled, y_train)

predictions = model.predict(x_test_scaled)
print("The forecast:", predictions)

Предсказания: [ 5382479.07544226  3756698.2081534   4942267.44785373 13921025.61154127
  6337828.18249274  1962713.99336096  3365844.72681006  6165157.36820965
  6779806.63380923  4190369.379108    6582013.11981285 10270227.77327529
  1557623.75780602   947133.90484383   938692.78318183  6939363.66134602
  2258008.53321736  1426311.12001087  2852382.88886483  5349823.68280591
 10509740.32561656 15197642.71061241  6838778.89316698  5240725.30031945
  7275817.87971319  3764743.15105651  1380086.13022309  4155046.19738425
  7427436.006959    2154418.84417786  2699984.46937375  4131387.29620779
  1849473.71073711   640541.08988399  1230223.9391683   2156161.24784838
  1678122.09313385  6915240.12992705 13957595.97290649  3617698.57287892
  2841498.07130096  8772595.41202722  4841141.40951416  3001319.18018234
  2002816.59313255  2099958.31771246  5528921.63845616  4175170.33991179
   701681.77053407   755805.12490712  6652775.79436035 11933921.55572671
  5578732.40858955  9546123.92851404 

Part 2: Experiments with machine learning models.

**3 Balls**. Do experiments with the machine learning methods you have written. Separate the training and test samples at 0.8 and 0.2 respectively. Measure the MSE, MAE, RMSE error. Use the KNNRegressor and LinearRegression methods from the sklearn library, compare the quality of your solutions and library.

In [ ]:
# We have already identified the training and test samples, measure the MSE, MAE, RMSE error